In [ ]:
!pip install pymorphy2

In [2]:
import pandas as pd
import nltk
import pymorphy2
import re

nltk.download('punkt')
morph = pymorphy2.MorphAnalyzer()


# посмотрим на наши данные
df = pd.read_csv('test_data.csv')
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [3]:
# мы проверяем только то, что говорил менеджер, 
# поэтому сразу выделим только полезные данные

manager_df = df[df['role'] == 'manager']
manager_df = manager_df.reset_index() 
manager_df.head()

,index,dlg_id,line_n,role,text
0,1,0,1,manager,Алло здравствуйте
1,3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
2,5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
3,8,0,8,manager,Угу а на что вы обращаете внимание при выборе
4,11,0,11,manager,Что для вас приоритет


In [4]:
#отметим приветствия и прощания
def search_greetings_and_partings(df):
    greetings = []
    partings = []
    for index, row in df.iterrows():
        g_mark = False
        p_mark = False

        line_n = row['line_n']
        text = row['text']

        if re.search(r'\bздравствуйте\b', text) or re.search(r'\bдобрый\b', text):
            g_mark = True
        
        elif re.search(r'\bсвидания\b', text) or re.search(r'\bвсего доброго\b', text):
            p_mark = True
     
        greetings.append(g_mark)
        partings.append(p_mark)


    return greetings, partings


greetings, partings = search_greetings_and_partings(manager_df)

manager_df['greeting'] = greetings
manager_df['parting'] = partings

manager_df.head()

,index,dlg_id,line_n,role,text,greeting,parting
0,1,0,1,manager,Алло здравствуйте,True,False
1,3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False
2,5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,False,False
3,8,0,8,manager,Угу а на что вы обращаете внимание при выборе,False,False
4,11,0,11,manager,Что для вас приоритет,False,False


In [5]:
#извлечем имена менеджеров
def search_name(df):
    names = {}
    introduced = []
    for index, row in df.iterrows():
        line_id = row['line_n']
        dlg_id = row['dlg_id']
        text = row['text']

        sentence = nltk.word_tokenize(text)
        
        for i in range(1, len(sentence) - 1):
            prev_word = sentence[i-1]
            word = sentence[i].lower()
            next_word = sentence[i+1]

            p = morph.parse(word)[0]

            if p.normal_form == 'звать':
                prev_score = [1 for p in morph.parse(prev_word) if 'Name' in p.tag]
                next_score = [1 for p in morph.parse(next_word) if 'Name' in p.tag]
        
                if prev_score > next_score:
                    names[dlg_id] = prev_word
                else:
                    names[dlg_id] = next_word
                introduced.append(text)
                
            elif (word == 'это' or word =='я') and [1 for p in morph.parse(next_word) if 'Name' in p.tag]:

                    names[dlg_id] = next_word
                    introduced.append(text)


    return names, introduced


names, intrs = search_name(manager_df)

print(names)
print('\nМенеджер представил себя в {} диалогах'.format(len(names)))


num_dlg = len(manager_df['dlg_id'].value_counts().values.tolist())
print('\nНа самом деле здесь {} диалогов'.format(num_dlg))

{0: 'ангелина', 1: 'ангелина', 2: 'ангелина', 3: 'максим', 5: 'анастасия'}

Менеджер представил себя в 5 диалогах

На самом деле здесь 6 диалогов


In [6]:
#реплики, где менеджер представился
for intr in intrs:
    print(intr)
    print('\n')

Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается


Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается


Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там


Добрый меня максим зовут компания китобизнес удобно говорить


Да это анастасия




In [7]:
#извлечем названия компаний
def search_company(df):
    company_names = {}
    for index, row in df.iterrows():
        dlg_id = row['dlg_id']
        text = row['text']
        sentence = nltk.word_tokenize(text)

        for i in range(len(sentence)-1):
          word = sentence[i]
          next_word = sentence[i+1]

          p = morph.parse(word)[0]
          if p.normal_form == 'компания' and [1 for p in morph.parse(next_word) if 'NOUN' in p.tag]:
              company_names[dlg_id] = next_word

    return company_names

print(' Company_names: ', search_company(manager_df))

 Company_names:  {0: 'диджитал', 1: 'диджитал', 2: 'диджитал', 3: 'китобизнес'}


In [8]:
check = {}

for i in range(num_dlg):
    gr_mark = False
    pr_mark = False
    for index, row in manager_df[manager_df.dlg_id == i].iterrows():
        dlg_id = row['dlg_id']
        gr = row['greeting']
        pr = row['parting']
        if gr:
            gr_mark = True

        elif pr:
            pr_mark = True
    if gr_mark and pr_mark:
        check[i] = True
    else:
        check[i] = False

print('Информация о выполненных требованиях в диалоге с клиентом: ', check)

Информация о выполненных требованиях в диалоге с клиентом:  {0: True, 1: True, 2: False, 3: True, 4: False, 5: False}
